In [4]:
import tensorflow as tf

from tensorflow.keras import layers, models, applications



import tensorflow as tf

from tensorflow.keras import datasets

from tensorflow.keras.utils import to_categorical





class SpatialAttention(layers.Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.conv = layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        return self.conv(concat)

# Define the CBAM block
def cbam_block(input_feature, ratio=8):
    # # Channel Attention Module
    # channel = input_feature.shape[-1]
    # avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    # avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    # avg_pool = layers.Dense(channel // ratio, activation='relu')(avg_pool)
    # avg_pool = layers.Dense(channel, activation='sigmoid')(avg_pool)

    # max_pool = layers.GlobalMaxPooling2D()(input_feature)
    # max_pool = layers.Reshape((1, 1, channel))(max_pool)
    # max_pool = layers.Dense(channel // ratio, activation='relu')(max_pool)
    # max_pool = layers.Dense(channel, activation='sigmoid')(max_pool)

    # channel_attention = layers.Add()([avg_pool, max_pool])
    # channel_attention = layers.Activation('sigmoid')(channel_attention)
    # channel_refined = layers.Multiply()([input_feature, channel_attention])

    # Spatial Attention Module
    spatial_attention = SpatialAttention()(input_feature)
    refined_feature = layers.Multiply()([input_feature, spatial_attention])
    return refined_feature


def depthwise_separable_conv_block(inputs, filters, strides=1):

    x = layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(inputs)

    x = layers.BatchNormalization()(x)

    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size=1, strides=1, padding='same')(x)

    x = layers.BatchNormalization()(x)

    x = layers.ReLU()(x)

    return x




def FD_MobileNet(input_shape=(64, 64, 3), num_classes=10):

    inputs = layers.Input(shape=input_shape)



    # Initial Convolution Block

    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x=cbam_block(x)

    # Depthwise Separable Convolution Blocks

    x = depthwise_separable_conv_block(x, filters=64, strides=1)
    x = depthwise_separable_conv_block(x, filters=128, strides=2)
    # x = depthwise_separable_conv_block(x, filters=128, strides=1)
    x = depthwise_separable_conv_block(x, filters=256, strides=2)
    x = depthwise_separable_conv_block(x, filters=256, strides=1)
    x = depthwise_separable_conv_block(x, filters=512, strides=2)



    # Additional layers as per FD MobileNet structure

    for _ in range(5):

        x = depthwise_separable_conv_block(x, filters=512, strides=1)



    x = depthwise_separable_conv_block(x, filters=1024, strides=2)


    distillation_feature = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(128, activation='relu')(distillation_feature)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name='FD_MobileNet')

    return model

fd_mobilenet = FD_MobileNet(input_shape=(64, 64, 3), num_classes=10)

fd_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

fd_mobilenet.summary()
print(len(fd_mobilenet.layers))

Model: "FD_MobileNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 32, 32,    │        896 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_18[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_30 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_attention_2 │ (None, 32, 32, 1) │         99 │ re_lu_30[0][0]    │
│ (SpatialAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_2          │ (None, 32, 32,    │          0 │ re_lu_30[0][0],   │
│ (Multiply)          │ 32)               │            │ spatial_attentio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_14 │ (None, 32, 32,    │        320 │ multiply_2[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_31 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 32, 32,    │      2,112 │ re_lu_31[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_32 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_15 │ (None, 16, 16,    │        640 │ re_lu_32[0][0]    │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_33 (ReLU)     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 16, 16,    │      8,320 │ re_lu_33[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_21[0][0] 

 Total params: 1,469,933 (5.61 MB)

 Trainable params: 1,458,797 (5.56 MB)

 Non-trainable params: 11,136 (43.50 KB)

57


In [5]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

num_classes = 10

x_train = tf.image.resize(x_train, (64, 64))

x_test = tf.image.resize(x_test, (64, 64))


x_train = tf.cast(x_train, tf.float32) / 255.0

x_test = tf.cast(x_test, tf.float32) / 255.0


y_train = to_categorical(y_train, num_classes)

y_test = to_categorical(y_test, num_classes)




fd_mobilenet = FD_MobileNet(input_shape=(64, 64, 3), num_classes=num_classes)




fd_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



history = fd_mobilenet.fit(

    x_train, y_train,

    validation_data=(x_test, y_test),

    batch_size=16,

    epochs=20,

    verbose=1

)




test_loss, test_accuracy = fd_mobilenet.evaluate(x_test, y_test)

print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")


Epoch 1/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - accuracy: 0.3519 - loss: 1.7599 - val_accuracy: 0.4728 - val_loss: 1.5282
Epoch 2/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.5959 - loss: 1.1570 - val_accuracy: 0.6525 - val_loss: 1.0019
Epoch 3/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.6895 - loss: 0.8880 - val_accuracy: 0.6956 - val_loss: 0.8743
Epoch 4/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.7499 - loss: 0.7208 - val_accuracy: 0.7251 - val_loss: 0.7919
Epoch 5/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.7956 - loss: 0.5944 - val_accuracy: 0.7082 - val_loss: 0.8796
Epoch 6/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8282 - loss: 0.4998 - val_accuracy: 0.7354 - val_loss: 0.8022
Epoch 7/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8479 - loss: 0.4401 - val_accuracy: 0.7502 - val_loss: 0.7708
Epoch 8/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.8730 - loss: 0

## saved the model

In [3]:
fd_mobilenet.save("fd_mobilenet_model_spatial_attention.h5")


## transfer learning
the last two fully connected layers from the above model are no freezed and the rest is freezed.


In [8]:
fd_mobilenet = tf.keras.models.load_model("/kaggle/input/mode-fd-spatial/fd_mobilenet_model_spatial_attention.h5",custom_objects={'SpatialAttention': SpatialAttention})
# model = load_model("path_to_your_model.h5", custom_objects={'SpatialAttention': SpatialAttention})
# 
for layer in fd_mobilenet.layers[:-2]:
    layer.trainable = False
fd_mobilenet.summary()

Model: "FD_MobileNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_attention_1 │ (None, 32, 32, 1) │         99 │ re_lu_15[0][0]    │
│ (SpatialAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 32, 32,    │          0 │ re_lu_15[0][0],   │
│ (Multiply)          │ 32)               │            │ spatial_attentio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_7  │ (None, 32, 32,    │        320 │ multiply_1[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_16 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 32, 32,    │      2,112 │ re_lu_16[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_17 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_8  │ (None, 16, 16,    │        640 │ re_lu_17[0][0]    │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 16, 16,    │      8,320 │ re_lu_18[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_12[0][0] 

 Total params: 1,198,063 (4.57 MB)

 Trainable params: 132,490 (517.54 KB)

 Non-trainable params: 1,065,571 (4.06 MB)

 Optimizer params: 2 (12.00 B)

In [9]:
x = fd_mobilenet.layers[-3].output  # Second to last layer
x = tf.keras.layers.Dense(512, activation='relu')(x)  # New dense layer
x = tf.keras.layers.Dense(3, activation='softmax')(x)  # Adjust for your number of classes (e.g., 3 classes)

# Create the new model with updated final layers
new_model = tf.keras.Model(inputs=fd_mobilenet.input, outputs=x)

# Compile the model (you can use a different optimizer or learning rate if needed)
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

new_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_attention_1 │ (None, 32, 32, 1) │         99 │ re_lu_15[0][0]    │
│ (SpatialAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 32, 32,    │          0 │ re_lu_15[0][0],   │
│ (Multiply)          │ 32)               │            │ spatial_attentio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_7  │ (None, 32, 32,    │        320 │ multiply_1[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_16 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 32, 32,    │      2,112 │ re_lu_16[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_17 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_8  │ (None, 16, 16,    │        640 │ re_lu_17[0][0]    │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 16, 16,    │      8,320 │ re_lu_18[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_12[0][0] 

 Total params: 1,591,910 (6.07 MB)

 Trainable params: 526,339 (2.01 MB)

 Non-trainable params: 1,065,571 (4.06 MB)

In [10]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming dataset is extracted to '/kaggle/working/'
dataset_folder = '/kaggle/input/trashnet'

# Create ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = os.path.join(dataset_folder, 'train')
test_dir = os.path.join(dataset_folder, 'valid')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Resize images to match model input
    batch_size=32,  # Number of samples per batch
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset="training"  # Use the training subset (80% of the data)
)

# Load images for validation (use subset="validation" for validation data)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Resize images to match model input
    batch_size=32,  # Number of samples per batch
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset="validation"  # Use the validation subset (20% of the data)
)


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 981 images belonging to 3 classes.
Found 243 images belonging to 3 classes.
Found 162 images belonging to 3 classes.


In [11]:
# Assuming new_model is already defined
dummy_input = tf.random.normal([1, 64, 64, 3])  # Example input size for MobileNet
new_model(dummy_input)


<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.1928833 , 0.50394243, 0.30317423]], dtype=float32)>

In [12]:
new_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 337ms/step - accuracy: 0.5044 - loss: 0.9954 - val_accuracy: 0.5597 - val_loss: 1.0573
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.7227 - loss: 0.6725 - val_accuracy: 0.5473 - val_loss: 1.0381
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.7791 - loss: 0.5403 - val_accuracy: 0.5885 - val_loss: 0.9860
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8091 - loss: 0.4872 - val_accuracy: 0.6132 - val_loss: 1.0176
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8474 - loss: 0.4179 - val_accuracy: 0.6420 - val_loss: 0.9740
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8990 - loss: 0.3223 - val_accuracy: 0.6173 - val_loss: 0.9700
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9372 - loss: 0.2689 - val_accuracy: 0.5926 - val_loss: 1.0546
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.9555 - loss: 0.2236 - val_accuracy: 0.5802 - val_loss: 

In [13]:
test_loss, test_accuracy = new_model.evaluate(test_generator)

print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 351ms/step - accuracy: 0.7809 - loss: 0.7838
Test loss: 0.7975415587425232, Test accuracy: 0.7716049551963806


In [14]:
new_model.save("transfer_fd_spatial.h5")


## finetuning 
unfreezing more layers

In [16]:
fd_mobilenet = tf.keras.models.load_model("/kaggle/input/mode-fd-spatial/fd_mobilenet_model_spatial_attention.h5",custom_objects={'SpatialAttention': SpatialAttention})
# for layer in fd_mobilenet.layers[:-10]:
#     layer.trainable = False
# fd_mobilenet.summary()

In [22]:
x = fd_mobilenet.layers[-3].output  # Second to last layer
x = tf.keras.layers.Dense(512, activation='relu')(x)  # New dense layer
x = tf.keras.layers.Dense(3, activation='softmax')(x)  # Adjust for your number of classes (e.g., 3 classes)

# Create the new model with updated final layers
fine_tuned_model = tf.keras.Model(inputs=fd_mobilenet.input, outputs=x)
for layer in fine_tuned_model.layers[:-20]:
    layer.trainable = False
fine_tuned_model.summary()
# Compile the model (you can use a different optimizer or learning rate if needed)


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │        896 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_attention_1 │ (None, 32, 32, 1) │         99 │ re_lu_15[0][0]    │
│ (SpatialAttention)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 32, 32,    │          0 │ re_lu_15[0][0],   │
│ (Multiply)          │ 32)               │            │ spatial_attentio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_7  │ (None, 32, 32,    │        320 │ multiply_1[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_16 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 32, 32,    │      2,112 │ re_lu_16[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_17 (ReLU)     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_8  │ (None, 16, 16,    │        640 │ re_lu_17[0][0]    │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_18 (ReLU)     │ (None, 16, 16,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 16, 16,    │      8,320 │ re_lu_18[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        512 │ conv2d_12[0][0] 

 Total params: 1,591,910 (6.07 MB)

 Trainable params: 1,324,547 (5.05 MB)

 Non-trainable params: 267,363 (1.02 MB)

In [23]:
fine_tuned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [24]:
dummy_input = tf.random.normal([1, 64, 64, 3])  # Example input size for MobileNet
fine_tuned_model(dummy_input)


<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.24017987, 0.46214727, 0.2976729 ]], dtype=float32)>

In [25]:
fine_tuned_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 160ms/step - accuracy: 0.8167 - loss: 0.4805 - val_accuracy: 0.6543 - val_loss: 1.3959
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.9942 - loss: 0.0378 - val_accuracy: 0.6296 - val_loss: 1.7933
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.9968 - loss: 0.0131 - val_accuracy: 0.6296 - val_loss: 1.7964
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9961 - loss: 0.0106 - val_accuracy: 0.6337 - val_loss: 1.9294
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9991 - loss: 0.0049 - val_accuracy: 0.6255 - val_loss: 1.9841
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9997 - loss: 0.0041 - val_accuracy: 0.6296 - val_loss: 1.9627
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.6420 - val_loss: 1.9839
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9997 - loss: 0.0012 - val_accuracy: 0.6337 -

In [26]:
test_loss, test_accuracy = fine_tuned_model.evaluate(test_generator)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.7716 - loss: 1.2822
Test loss: 1.1394274234771729, Test accuracy: 0.7777777910232544


In [27]:
fine_tuned_model.save("fine_tuned_fd_12_spatial.h5")

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/fine_tuned_fd_12_spatial.h5", custom_objects={'SpatialAttention': SpatialAttention})

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Enable optimizations (recommended for Raspberry Pi)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # Float16 quantization

# Convert the model
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open("fine_tuned_fd_12_spatial.tflite", "wb") as f:
    f.write(tflite_model)
